# Пакеты

## Пути импортирования

Каждый пакет идентифицируется уникальной строкой, которая называется **путь импортирования**(import path). Пути импортирования указываются в объявлении import:

In [ ]:
import (
    "fmt"
    "math/rand"
    "encoding/json"
    "golang.org/x/net/html"
    "github.com/go-sql-driver/mysql"   
)

Спецификация языка Go **не определяет** смысл этих строк или как найти в файловой системе путь до пакета. Эти вопросы оставляются на решение инструментам(tools). В этой главе мы рассмотрим как инструмент go tool интерпретирует эти строки, т.к. именно его и используют большинство программистов на Go для сборки, тестирования и т.д. Но надо знать, что есть и другие инструменты, у которых собственный механизм именования, поиска пакетов, выполнения тестов и т.д.

Пакеты, которые будут передаваться другим лицам или будут доступны публично **должны иметь** уникальный глобальный путь импортирования. Для этого все пакеты не из стандартной библиотеки должны начинаться с доменного имени организации, которая владеет этим пакетом или предоставляет к нему доступ(hosting). Это также упрощает поиск пакета. Например, в объявлениях приведённых наверху импортируется пакет для парсинга html, который поддерживается командой, работающей над языком Go и также импортируется сторонний драйвер базы данных MySQL.

## Объявление пакета

Вначале каждого исходного файла Go необходимо добавлять объявление пакета. Основной целью этого объявления является определение идентификатора по-умолчанию для этого пакета(его ещё называют **именем пакета**) при импортировании другим пакетом. Например, каждый файл пакета math/rand начинается с строки "package rand". Поэтому после импорта этого пакета можно получить доступ к его элементам через rand: rand.Int, rand.Float64 и т.д.:

In [3]:
import (
    "fmt"
    "math/rand"
)

fmt.Sprint(rand.Int())

6963677671731025125


Принято такое соглашение, что имя пакета - это **последний сегмент** пути импортирования и в результате может быть такой случай, что два пакета имеют одинаковое имя. Например, пакеты с путями импортирования math/rand и crypto/rand имеют имя rand. Позже мы рассмотрим как можно использовать оба пакета в одной программе.

Есть также три исключения из описанного соглашения последнего сегмента:
1. Пакет, который определяет команду(запускаемую Go программу), всегда имеет имя main независимо от пути импортирования.
2. Имя пакета некоторых файлов в папке пакета может оканчиваться на \_test, если имя этого файла в файловой системе оканчивается на \_test.go. Такие папки могут определять два пакета: обычный и ещё один пакет, который называется внешним пакетом тестирования(external test package). Суффикс \_test указывает команде go test, что необходимо собирать оба пакета, а также какой файл принадлежит какому пакету.
3. Некоторые инструменты для управления зависимостями добавляют версию пакета в конец пути импортирования, как например, "gopkg.io/yaml.v2". Этот суффикс не входит в имя пакета, т.е. в нашем примере именем пакета будет просто yaml.

## Импортирование пакета

Исходный файл Go может содержать 0 или больше объявлений import сразу после объявления package, но до какого-либо не import объявления. Каждое объявление import может определять путь имфортирования для одного пакета или нескольких пакетов списком в скобках. Два объявления import ниже являются эквивалентными, но второй способ встречается чаще:

In [ ]:
import "fmt"
import "os"

import (
    "fmt"
    "os"
)

Импортируемые пакеты можно группировать с помощью пустой строки между группами. Порядок путей импортирования внутри группы не имеет значения, но по принятому соглашению, их сортируют(gofmt и goimports также выполняют сортировку):

In [ ]:
import (
    "fmt"
    "html/template"
    "os"

    "golang.org/x/net/html"
    "golang.org/x/net/ipv4"
)

Если нам необходимо импортировать два пакета с одинаковым именем, как например, math/rand и crypto/rand, в третий пакет, то мы должны определить **альтернативное имя** хотя бы для одного пакета, чтобы избежать конфликта имён. Такой импорт называется **переименующим импортом**(renaming import):

In [ ]:
import (
    "crypto/rand"
    mrand "math/rand" // Альтернативное имя.
)

Альтернативное имя влияет только на файл, где есть такой импорт. Другие файлы и даже файлы из этого же пакета, могут использовать имя по-умолчанию или использовать другое альтернативное имя.

Каждое объявление import устанавливает зависимость между текущим и импортируемым пакетом. Команда go build выведет ошибку, если зависимости образуют цикл.

## Пустые импорты

Если импортировать пакет, но не использовать какой-либо символ из этого пакета, то будет ошибка компиляции. Но иногда нам нужно импортировать пакет только для того, чтобы сработал побочный эффект импортирования: инициализировались всё переменные уровня пакета и выполнилась функция init. При этом чтобы избежать ошибки неиспользования какого-либо символа из пакета нам необходимо использовать переименующий импорт, где альтернативным именем будет _ - пустой идентификатор. Как обычно к пустому идентификатор никогда нельзя обращаться.

In [ ]:
import _ "image/png"

Такой вид импорта известен как **пустой импорт**(blank import). Пустые импорты наиболее часто используются для реализации механизма времени компиляции, с помощью которого основная программа может активировать или выключить дополнительный функционал.